In [4]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Tue Oct 31 20:44:46 2023

@author: michal
"""

import numpy as np
from queue import Queue
from scipy.spatial import ConvexHull, Delaunay
from scipy.spatial.distance import euclidean
from scipy.stats.qmc import PoissonDisk
from sklearn.feature_selection import RFECV
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from sklearn.decomposition import PCA, KernelPCA, SparsePCA, TruncatedSVD
from sklearn.metrics import r2_score
from boruta import BorutaPy
import pycaret.regression as py_reg
import pycaret.classification as py_clf
from imblearn.over_sampling import SMOTE
import torch
from torch import nn, optim
from torch.utils.data import DataLoader, Dataset
import pytorch_lightning as pl
import pandas as pd
import numpy as np
from collections import deque
import umap
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import numba
from pycaret.regression import *
import lightgbm as lgb
from scipy import *
from sklearn import datasets


iris = datasets.load_iris()
data = iris.data, iris.feature_names
df = pd.DataFrame(iris.data, columns=iris.feature_names)



mapper = umap.UMAP(n_components=3, metric="minkowski", n_neighbors=20, min_dist=1.0)
mapping = mapper.fit_transform(df)



hull = ConvexHull(mapping)

for simplex in hull.simplices:
    plt.plot(mapping[simplex, 0], mapping[simplex, 1], 'k-')

def poisson_disc_sampling(points, radius=0.1):
    # Generate the Poisson disk samples within the convex hull
    tri = Delaunay(points)
    lb, ub = tri.min_bound, tri.max_bound
    size = np.abs(ub - lb)
    poisson = PoissonDisk(d=3, radius=radius)
    unit_samples = poisson.fill_space()
    samples = lb + size * unit_samples

    # Select the points that are within the convex hull
    simplex = tri.find_simplex(samples)
    samples = samples[simplex >= 0]
    return samples


# Generate random points in 3D space
points = mapping

# Compute the convex hull
hull = ConvexHull(points)

# Compute the Delaunay triangulation of the hull
tri = Delaunay(points)

# generate poisson disk samples within the convex hull
samples = poisson_disc_sampling(points, radius=0.1)

inverse_samples = mapper.inverse_transform(samples)
data_new = pd.concat([df, pd.DataFrame(inverse_samples, columns=df.columns)]).reset_index(drop=True)

print(data_new.shape, df.shape)

py_reg.setup(df,
             target="petal width (cm)",
             preprocess=True,
             feature_selection=False,
             feature_selection_method="sequential",
             n_features_to_select=20,
             normalize=False,
             normalize_method="robust",
             remove_multicollinearity=True,
             multicollinearity_threshold=0.95,
             low_variance_threshold=0.1,
             pca=False,
             pca_method="linear",
             pca_components=6,
             profile=False)

best = py_reg.compare_models()


(298, 4) (150, 4)


Traceback (most recent call last):
  File "/Users/michal/opt/anaconda3/lib/python3.9/site-packages/pycaret/internal/logging.py", line 144, in create_logger
    ch = logging.FileHandler(path)
  File "/Users/michal/opt/anaconda3/lib/python3.9/logging/__init__.py", line 1146, in __init__
    StreamHandler.__init__(self, self._open())
  File "/Users/michal/opt/anaconda3/lib/python3.9/logging/__init__.py", line 1175, in _open
    return open(self.baseFilename, self.mode, encoding=self.encoding,
OSError: [Errno 30] Read-only file system: '/logs.log'


,Description,Value
0,Session id,5846
1,Target,petal width (cm)
2,Target type,Regression
3,Original data shape,"(150, 4)"
4,Transformed data shape,"(150, 4)"
5,Transformed train set shape,"(105, 4)"
6,Transformed test set shape,"(45, 4)"
7,Numeric features,3
8,Preprocess,True
9,Imputation type,simple


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
rf,Random Forest Regressor,0.1442,0.0327,0.1787,0.9403,0.0810,0.2311,0.0270
knn,K Neighbors Regressor,0.1383,0.0333,0.1793,0.9399,0.0808,0.2118,0.0060
ada,AdaBoost Regressor,0.1441,0.0340,0.1822,0.9383,0.0779,0.2032,0.0070
lightgbm,Light Gradient Boosting Machine,0.1471,0.0341,0.1828,0.9378,0.0817,0.2222,0.0090
lar,Least Angle Regression,0.1442,0.0368,0.1855,0.9364,0.0853,0.2433,0.0050
huber,Huber Regressor,0.1454,0.0369,0.1859,0.9364,0.0856,0.2444,0.0060
lr,Linear Regression,0.1442,0.0368,0.1855,0.9364,0.0853,0.2433,0.0050
br,Bayesian Ridge,0.1445,0.0368,0.1856,0.9363,0.0851,0.2418,0.0050
ridge,Ridge Regression,0.1456,0.0371,0.1866,0.9358,0.0850,0.2385,0.0060
et,Extra Trees Regressor,0.1507,0.0376,0.1929,0.9298,0.0866,0.2289,0.0230


Processing:   0%|          | 0/85 [00:00<?, ?it/s]

In [5]:
py_reg.setup(data_new,
             target="petal width (cm)",
             preprocess=True,
             feature_selection=False,
             feature_selection_method="sequential",
             n_features_to_select=20,
             normalize=False,
             normalize_method="robust",
             remove_multicollinearity=True,
             multicollinearity_threshold=0.95,
             low_variance_threshold=0.1,
             pca=False,
             pca_method="linear",
             pca_components=6,
             profile=False)

best = py_reg.compare_models()


Traceback (most recent call last):
  File "/Users/michal/opt/anaconda3/lib/python3.9/site-packages/pycaret/internal/logging.py", line 144, in create_logger
    ch = logging.FileHandler(path)
  File "/Users/michal/opt/anaconda3/lib/python3.9/logging/__init__.py", line 1146, in __init__
    StreamHandler.__init__(self, self._open())
  File "/Users/michal/opt/anaconda3/lib/python3.9/logging/__init__.py", line 1175, in _open
    return open(self.baseFilename, self.mode, encoding=self.encoding,
OSError: [Errno 30] Read-only file system: '/logs.log'


,Description,Value
0,Session id,4537
1,Target,petal width (cm)
2,Target type,Regression
3,Original data shape,"(298, 4)"
4,Transformed data shape,"(298, 4)"
5,Transformed train set shape,"(208, 4)"
6,Transformed test set shape,"(90, 4)"
7,Numeric features,3
8,Preprocess,True
9,Imputation type,simple


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
knn,K Neighbors Regressor,0.1174,0.0258,0.1589,0.9519,0.0727,0.3058,0.0060
et,Extra Trees Regressor,0.1209,0.0262,0.1598,0.9515,0.0749,0.3149,0.0210
rf,Random Forest Regressor,0.1223,0.0263,0.1602,0.9511,0.0740,0.3121,0.0270
gbr,Gradient Boosting Regressor,0.1224,0.0265,0.1609,0.9506,0.0735,0.3166,0.0080
ada,AdaBoost Regressor,0.1249,0.0267,0.1612,0.9496,0.0721,0.3384,0.0090
lightgbm,Light Gradient Boosting Machine,0.1236,0.0276,0.1642,0.9477,0.0745,0.3125,0.0060
xgboost,Extreme Gradient Boosting,0.1305,0.0302,0.1725,0.9437,0.0824,0.3097,0.0080
lar,Least Angle Regression,0.1373,0.0325,0.1775,0.9402,0.0805,0.3642,0.0060
lr,Linear Regression,0.1373,0.0325,0.1775,0.9402,0.0805,0.3642,0.0050
br,Bayesian Ridge,0.1373,0.0325,0.1775,0.9401,0.0804,0.3642,0.0050


Processing:   0%|          | 0/85 [00:00<?, ?it/s]